# modelarts前三期总结

接触modelarts时间也不短了，现对前三期做个总结。前三期主要是以猫狗识别为例的图像分类实验，第一期包括两个案例，第一期带我们初识modelarts，第二期主要教我们如何提高准确度，第三期带我们更加深入的了解如何通过调参优化器来提高准确度。

## 自动学习

对于新注册用户，需要先实名认证，生成访问秘钥，点击右上角的账户中心->基本信息->管理凭证->新增访问秘钥，按照步骤生成后，来到modelarts网址，点击全局配置，添加访问秘钥即可。具体步骤可参见第一期相关链接。大概步骤: 

[图片1](./图片1.png)

创建图像分类项目，创建完成后下载数据集并解压上传到obs桶内，该数据集包含了四种不同的花卉，它会自动跳转到数据标注截面，对未标注的图片进行标注，标注完成后进入已标注界面，检查是否每种标签正确，数据标注完成后进行模型训练，时长设为0.1h，然后点击开始训练，大概6分钟后训练完成，可以查看下训练精度，部署上线

 [图片2](./图片2.png)
 
等待部署成功成功后，上传图片后进行预测。通过结果可看到其准确度较高，结果可靠，并且过程易懂，页面友好性较好。

## 猫狗识别

第一期另一个案例是在notebook环境下进行猫狗识别，首先要创建相关配置的notebook，参数如下:
计费方式	按需计费
名称	Notebook实例名称，如 image_recognition
工作环境	Python3
资源池	选择"公共资源池"即可
类型	本案例使用较为复杂的深度神经网络模型，需要较高算力，选择"GPU"
规格	选择"8核 | 64GiB | 1*p100"
存储配置	选择EVS，磁盘规格5GB
配置好以后创建notebook并打开如图:

[图片3](./图片3.png)

在此编写代码即可，主要步骤分为导入依赖库、下载数据包、读取数据、数据标记、分割、构建神经网络、模型训练、保存模型、评估模型准确度。我们可以通过调参或更改算法模型来提高准确度。调参，可以调节learning_rate、epoch和patch三个参数来提高准确度。通过调节发现learning_rate=1e-4, patch=16，epoch=30即可。除了调参还可以修改算法模型，如使用resnet50算法，不过用该算法时需要冻结base_model所有层，添加全链接分类层 

[图片4](./图片4.png)

尺寸默认设置为224*224，我提高准确度采用的模型是xception算法，通过测试发现该算法尺寸299*299要比224*224准确度高，不过训练速度却降低，使用该算法如果冻结base_model层，发现模型输出时四个参数有三个为none，导致在后来的模型训练时，测试集丢失率很高达到7.0之上，准确度仅仅0.5多。可见xception算法需要带着imagenet权重进行训练。不可冻结，这也算是它在调用时与resnet50不同点。x=base_model.output代表模型输出为二维，这行代码切记仅此一行，否则会导致多维输出。xception算法训练需要时间比较长，大概多半个小时，可能久了点。

## 图像分类综合应用

第二期具体步骤参考github，下载数据集、解压到本地、上传到obs桶。然后创建数据集，进行标注，每种标注20-30即可，之后进行智能标注，可能有几次会失败，原因是多方面的，很多是因为用户较多，多试几次，智能标注完成后进行人工确认。对于标注错误的可以在未标注截面，进行人工标注，迭代智能标注过程，完成最后标注。

[图片5](./图片5.png)

接下来是猫狗识别的模型训练及部署上线。首先下载训练数据集到本地，直接上传到ob桶，

[图片6](./图片6.png)

按照如上图设置好后，点击创建，创建完成后等待作业训练完成，运行成功后查看评估结果，创建tensorbroad查看loss， accuracy等参数的变化信息。记住在使用完以后要及时停止tensorbroad，在tensorbroad上我们可以清楚的看到oss、accuracy等评估指标的变化曲线，为开发者优化模型精度提供依据。我可以通过这个训练过程来调节参数来提高准确度。之后就是模型部署，上线预测。具体步骤参考github官网上modelarts截面。

## 模型参数&网络调优

第三期分为4个小任务，0. epochs和callbacks：介绍训练轮数，拟合问题以及Keras模型训练时的Callback回调函数; 1. 学习率和优化器：介绍学习率和不同优化器在模型训练中的实践;2. 数据增广：使用Keras DataGenerator预处理数据，实现数据增广;3. 使用预训练权重：使用预训练的参数权重文件进行迁移学习
任务0首先按照第一期notebook运行一遍，之后再重新训练5轮，在Epoch数值较小时，出现了上一个训练中出现的欠拟合情况，模型没有很好收敛训练便结束了。在接下来的训练中，我们提高了epoch的值，通过训练结果我们可以看到模型逐渐收敛。但是epoch的值并非越大越好，过大的epoch值可能会导致过拟合现象。除了epoch还有Earlystopping早停法，检测模型的某一项指标，如果在指定步数中指标没有提升，则将模型训练停止。
任务1通过比较同样结构，同样数据集，相同训练轮数下不同优化器以及不同初始学习率下训练的值得出优化器与初始学习率对训练的影响。该实例通过分别使用rmsprop、adam和SVG。在模型训练过程中涉及到一个关键概念——学习率。学习率代表着模型学习的速率，学习率的值越大，损失函数的变化速度越快。较高的学习率大小可以使模型快速收敛，但是可能会导致模型在局部极小值周围震荡。
任务2在原论文中，作者使用了原始尺寸、较大尺寸和较小尺寸三种规格，并获得了一定的准确率的提升(具体请参考论文或VGG模型解析文章)。在Keras中，我们通过DataGenerator，对输入的图片进行更丰富的预处理，包括缩放、旋转、平移、水平翻转等，来探索数据增广对模型精度带来的提升。预处理并不会直接增加数据集的样本数量，而是让样本具备更加丰富的特征，理论上，可以让训练的模型获得更好的泛化性。
任务3使用ImageNet预训练模型权重，我们使用预置的VGG16模型，加载预训练的权重，保留VGG16的卷积网络结构，只在最后的全连接层进行调整，使用GlobalAveragePooling2D将7x7x512的卷积结果进行全局池化，减少训练参数，并加入softmax激活、output shape为2的全连接层进行二分类。设置callback并训练模型，使用预训练的权重，可以较快提高模型精度。在高性能GPU环境下，每一轮需要几分钟左右，请实践者保持耐心，笔者的实践中训练到第11轮就达到了约90%的精度，耗时约8分钟。后续训练精度提升开始变慢，在41轮达到约92%的精度，耗时约30分钟。完成后进行模型保存，开始训练所有参数，仅需要训练三轮就可以达到95％的准确度。
[图片](./图片7.png)